In [5]:
#import packages
import random
import pickle
import numpy as np
import time

# PLAYING, TRAINING AND TESTING GAME

In this cell we define the parameters

In [16]:
learning_rate=0.1
decay_gamma=0.9
exp_rate=0.3
starting_player='agent'
init_board=''
n=4
file_name = '61'
policy_player1='pol61'
list_of_winners=[]
end_game_big=[]

In [2]:
#constructs the game class
class seq_game:
    def __init__(self, n):
        self.n=n
        self.IsEnd=False
        self.Player=starting_player
        self.board=init_board
        self.agent_states=[]
        self.exp_rate=exp_rate
        self.lr = learning_rate  # should decrease as it continues to gain a larger knowledge base
        self.decay_gamma = decay_gamma  # big gamma means thinking long term
        self.states_value = {}  # dict for storing {board, weight}
        self.end_game_list=[0,0]
        self.winner=None
    def check_for_losing_move(self, board):
        """
        Checks whether adding 0 or 1 are losing moves
        :return returns a list of losing moves, i.e either [], [0], [1], [0,1]:
        """
        current_board=board
        list_of_losers=[]
        for i in [0,1]:
            possible_loser=current_board+str(i)
            substring_to_test=possible_loser[(-self.n):]
            if substring_to_test in current_board:
                list_of_losers.append(i)
        return list_of_losers
    def check_for_winning_move(self):
        """
        Assumes there are two non-losing moves. Checks whether one of these moves forces next player to lose
        :return:
        """
        winning_move=[]
        for i in [0,1]:
            temp_board=self.board+str(i)
            if len(self.check_for_losing_move(temp_board))==2:
                winning_move.append(i)
        return winning_move
    def change_player(self):
        """
        Function for changing current player
        :return: returns none
        """
        if self.Player=='agent':
            self.Player='dummy'
        else:
            self.Player='agent'
    def update_board(self, move):
        """
        updates board
        :param move is a string:
        :return: returns none
        """
        self.board+=move
    def reset(self):
        """
        Resets the game
        :return: returns none
        """
        self.board = init_board
        self.IsEnd = False
        self.Player = starting_player
        self.agent_states=[]
        self.winner=None
    def savePolicy(self):
        """
        function for saving states and weights for use for the trained agent
        :return: returns None
        """
        fw = open('pol' + file_name, 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()
    def feedReward(self, reward):
        """
        function for giving rewards to all board states from the finished game.
        :param reward: reward to give. 1 or 0.
        :return: returns None
        """
        for st in reversed(self.agent_states):  # goes through all saved board states of this game
            if self.states_value.get(
                    st) is None:  # if it's not already in the dictionary (of board states of ALL games)
                self.states_value[st] = 0  # initialise a value for the state
            self.states_value[st] += self.lr * (
                        self.decay_gamma * reward - self.states_value[st])  # update weight for each board state
    def loadPolicy(self, file):
        """
        function for loading states and weights for the trained agent.
        :param file:
        :return:
        """
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()
    def smartmove(self, current_board):
        """
        function for choosing an action. This is for player 1 already trained.
            :param positions: available moves to make
            :param current_board: board state
            :return: returns action to make
         """
        losing_moves = self.check_for_losing_move(current_board)
        if len(losing_moves) == 1:  # if only one non-losing move
            non_losing_move = list({0, 1} - set(losing_moves))
            move = str(non_losing_move[0])
        elif len(losing_moves) == 0:
            winning_moves = self.check_for_winning_move()
            if len(winning_moves) > 0:
                move = str(winning_moves[0])
            else:
                value_max = -999
                #print(self.board, current_board, 'board')
                for p in [0,1]:
                    next_board = current_board+str(p)
                    value = 0 if self.states_value.get(next_board) is None else self.states_value.get(next_board)
                    if value >= value_max:
                        value_max = value
                        move = str(p)
        else:
            move= None
        return move
    def normalmove(self):
        """
        :return: returns move
        """
        losing_moves = self.check_for_losing_move(self.board)
        if len(losing_moves) == 1:  # if only one non-losing move
            non_losing_move = list({0, 1} - set(losing_moves))
            move = str(non_losing_move[0])
        elif len(losing_moves) == 0:
            winning_moves = self.check_for_winning_move()
            if len(winning_moves) > 0:
                move = str(winning_moves[0])
            else:
                move = str(random.randint(0, 1))
        else:
            move= None
        return move
    def training_game_(self, rounds=1000):
        for i in range(rounds):
            #if i % 1000 == 0:
            #    print("Rounds {}".format(i))
            while not self.IsEnd:
                move=self.normalmove()
                if move is not None:
                    self.update_board(move)
                else:
                    self.IsEnd = True
                    if self.Player=='dummy':#if player 1 has won
                        self.feedReward(1)
                        self.savePolicy()
                        list_of_winners.append('agent')
                    else:
                        list_of_winners.append('dummy')
                        self.feedReward(0)
                        self.savePolicy()
                if self.Player == 'agent':
                    self.agent_states.append(self.board)
                self.change_player()
            self.reset() #DONT USE THIS
    def training_game(self, rounds=10):
        for i in range(rounds):
            while not self.IsEnd:
                if self.Player=='dummy':
                    move = self.normalmove()
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        move = self.normalmove()
                    else:
                        move=self.smartmove(self.board)
                if move is not None:
                    self.update_board(str(move))
                else:
                    self.IsEnd = True
                    if self.Player=='dummy':#if player 1 has won
                        self.feedReward(1)
                        self.savePolicy()
                        list_of_winners.append('agent')
                    else:
                        list_of_winners.append('dummy')
                        self.feedReward(-1)
                        self.savePolicy()
                if self.Player == 'agent':
                    self.agent_states.append(self.board)
                self.change_player()
            self.reset()
    def testing_game(self, rounds=1000):
        for i in range(rounds):
            while not self.IsEnd:
                #print('PLAYER', self.Player)
                losing_moves = self.check_for_losing_move(self.board)
                if len(losing_moves) == 1:  # if only one non-losing move
                    non_losing_move = list({0, 1} - set(losing_moves))
                    move = str(non_losing_move[0])
                elif len(losing_moves) == 0:
                    winning_moves = self.check_for_winning_move()
                    if len(winning_moves) > 0:
                        move = str(winning_moves[0])
                    else:
                        """HERE IS WHERE THE EXCITING STUFF HAPPENS"""
                        if self.Player=='dummy':
                            move = str(random.randint(0, 1))
                        else:
                            move=self.smartmove(self.board)
                self.update_board(move)
                if self.Player == 'agent':
                    self.agent_states.append(self.board)
                #print(self.board)
                if len(losing_moves) == 2:  # GAME OVER
                    self.IsEnd = True
                    #with open('Data', 'a') as f:  # saves winner
                    #    f.write(self.board+', ')
                    if self.Player == 'dummy':
                        list_of_winners.append('agent')
                    else:
                        list_of_winners.append('dummy')
                self.change_player()
            self.reset()
    def end_game_testing(self, rounds=1000):
        for i in range(rounds):
            while not self.IsEnd:
                losing_moves = self.check_for_losing_move(self.board)
                if len(losing_moves) == 1:  # if only one non-losing move
                    if self.Player=='dummy':
                        self.end_game_list.append('x')
                    else:
                        self.end_game_list.append('x')
                    non_losing_move = list({0, 1} - set(losing_moves))
                    move = str(non_losing_move[0])
                elif len(losing_moves) == 0:
                    winning_moves = self.check_for_winning_move()
                    if len(winning_moves) > 0:
                        move = str(winning_moves[0])
                        self.end_game_list.append('w')
                    else:
                        if self.Player=='dummy':
                            move = str(random.randint(0, 1))
                            self.end_game_list.append(0)
                        else:
                            move=self.smartmove(self.board)
                            self.end_game_list.append(0)
                self.update_board(move)
                if len(losing_moves) == 2:  # GAME OVER
                    self.IsEnd = True
                    if self.Player == 'dummy':
                        list_of_winners.append('agent')
                        self.winner='agent'
                    else:
                        list_of_winners.append('dummy')
                        self.winner='dummy'
                self.change_player()
            #print(self.end_game_agent)
            end_game_big.append((self.end_game_list, self.winner))
            self.reset()
            self.end_game_list=[0,0]

    def end_game_random(self, rounds=1000):
        for i in range(rounds):
            while not self.IsEnd:
                losing_moves = self.check_for_losing_move(self.board)
                if len(losing_moves) == 1:  # if only one non-losing move
                    if self.Player=='dummy':
                        self.end_game_list.append('x')
                    else:
                        self.end_game_list.append('x')
                    non_losing_move = list({0, 1} - set(losing_moves))
                    move = str(non_losing_move[0])
                elif len(losing_moves) == 0:
                    winning_moves = self.check_for_winning_move()
                    if len(winning_moves) > 0:
                        move = str(winning_moves[0])
                        self.end_game_list.append('w')
                    else:
                        if self.Player=='dummy':
                            move = str(random.randint(0, 1))
                            self.end_game_list.append(0)
                        else:
                            move=str(random.randint(0, 1))
                            self.end_game_list.append(0)
                self.update_board(move)
                if len(losing_moves) == 2:  # GAME OVER
                    self.IsEnd = True
                    if self.Player == 'dummy':
                        list_of_winners.append('agent')
                        self.winner='agent'
                    else:
                        list_of_winners.append('dummy')
                        self.winner='dummy'
                self.change_player()
            #print(self.end_game_agent)
            end_game_big.append((self.end_game_list, self.winner))
            self.reset()
            self.end_game_list=[0,0]
    def change_strategy(self, rounds=1000):
        """Changing strategy"""
        for i in range(rounds):
            agent_move=1
            while not self.IsEnd:
                if self.Player=='dummy':
                    losing_moves=self.check_for_losing_move(self.board)
                    move=self.normalmove()
                else:
                    losing_moves = self.check_for_losing_move(self.board)
                    if len(losing_moves) == 1:  # if only one non-losing move
                        non_losing_move = list({0, 1} - set(losing_moves))
                        move = str(non_losing_move[0])
                        agent_move=1-int(move)
                    elif len(losing_moves) == 0:
                        winning_moves = self.check_for_winning_move()
                        if len(winning_moves) > 0:
                            move = str(winning_moves[0])
                            agent_move=1-int(move)
                        else:
                            move = str(agent_move)
                            agent_move=1-int(move)
                    else:
                        move=None
                if move is None:  # GAME OVER
                    self.IsEnd = True
                    if self.Player == 'dummy':
                        list_of_winners.append('agent')
                    else:
                        list_of_winners.append('dummy')
                        losing_games.append(self.board)
                else:
                    self.update_board(move)
                self.change_player()
            self.reset()

In [14]:
play = seq_game(n)

Below is for training until policy has more than x percent winnings

In [15]:
play = seq_game(n)
rounds=1000
total_rounds=0
list_of_winners=[]
print(len(play.states_value))
start_time=time.time()
while list_of_winners.count('agent')*100/rounds <100:
    list_of_winners = []
    play.training_game(rounds)
    list_of_winners=[]
    play.loadPolicy(policy_player1)
    play.testing_game(rounds)
    total_rounds+=rounds
    print('agent wins', list_of_winners.count('agent')*100/rounds, 'percent of the test games, the Q-table has', len(play.states_value), 'entries after', total_rounds)
end_time=time.time()
print('it took', round(end_time-start_time,3), 'seconds to finish')

0
agent wins 78.9 percent of the test games, the Q-table has 1091 entries after 1000
agent wins 83.2 percent of the test games, the Q-table has 1489 entries after 2000
agent wins 83.9 percent of the test games, the Q-table has 1693 entries after 3000
agent wins 93.3 percent of the test games, the Q-table has 1848 entries after 4000
agent wins 90.1 percent of the test games, the Q-table has 2017 entries after 5000
agent wins 91.3 percent of the test games, the Q-table has 2102 entries after 6000
agent wins 82.3 percent of the test games, the Q-table has 2162 entries after 7000
agent wins 93.9 percent of the test games, the Q-table has 2240 entries after 8000
agent wins 97.8 percent of the test games, the Q-table has 2293 entries after 9000
agent wins 89.1 percent of the test games, the Q-table has 2333 entries after 10000
agent wins 89.5 percent of the test games, the Q-table has 2365 entries after 11000
agent wins 94.2 percent of the test games, the Q-table has 2396 entries after 12000

### Below is the code for testing the changing strategy

In [8]:
sequence_game=seq_game(14)
list_of_winners=[]
losing_games=[]
sequence_game.change_strategy(1000)
print('the agent wins', list_of_winners.count('agent')/len(list_of_winners)*100, 'percent of the games')

the agent wins 99.6 percent of the games


### Below is the code for testing different parameters

In [214]:
list_of_alphas=[0.01*i for i in range(1,20,4)]
list_of_gammas=[0.1*i for i in range(1,10,2)]
list_of_epsilons=[0.1*i for i in range(1,10,2)]
for alpha in list_of_alphas:
    for gamma in list_of_gammas:
        for epsilon in list_of_epsilons:
            play = seq_game(n)
            play.lr=alpha
            play.decay_gamma=gamma
            play.exp_rate=epsilon
            rounds=1000
            list_of_winners=[]
            start_time=time.time()
            while list_of_winners.count('agent')*100/rounds <100:
                list_of_winners = []
                play.training_game2(rounds)
                #print('agent wins', list_of_winners.count('agent')*100/rounds, 'percent of the training games')
                play.loadPolicy(policy_player1)
                list_of_winners=[]
                play.testing_game(rounds)
                #print('agent wins', list_of_winners.count('agent')*100/rounds, 'percent of the test games')
            end_time=time.time()
            print(round(end_time-start_time,3), alpha, gamma, epsilon)

57.656 0.01 0.1 0.1
11.173 0.01 0.1 0.30000000000000004
10.985 0.01 0.1 0.5
4.98 0.01 0.1 0.7000000000000001
12.027 0.01 0.1 0.9
49.362 0.01 0.30000000000000004 0.1
16.817 0.01 0.30000000000000004 0.30000000000000004
5.409 0.01 0.30000000000000004 0.5
4.384 0.01 0.30000000000000004 0.7000000000000001
8.05 0.01 0.30000000000000004 0.9
17.26 0.01 0.5 0.1
6.479 0.01 0.5 0.30000000000000004
6.88 0.01 0.5 0.5
12.927 0.01 0.5 0.7000000000000001
19.074 0.01 0.5 0.9
108.717 0.01 0.7000000000000001 0.1
11.085 0.01 0.7000000000000001 0.30000000000000004
9.066 0.01 0.7000000000000001 0.5
9.542 0.01 0.7000000000000001 0.7000000000000001
4.725 0.01 0.7000000000000001 0.9
108.665 0.01 0.9 0.1
11.86 0.01 0.9 0.30000000000000004
6.692 0.01 0.9 0.5
3.273 0.01 0.9 0.7000000000000001
4.411 0.01 0.9 0.9
41.52 0.05 0.1 0.1
2.074 0.05 0.1 0.30000000000000004
2.904 0.05 0.1 0.5
34.676 0.05 0.1 0.7000000000000001
24.338 0.05 0.1 0.9
12.844 0.05 0.30000000000000004 0.1
3.52 0.05 0.30000000000000004 0.300000000

1.23 0.17 0.1 0.5
1.44 0.13 0.7000000000000001 0.30000000000000004
0.713 0.09 0.5 0.5
1.246 0.09 0.30000000000000004 0.5
1.247 0.09 0.1 0.5